## Final Project - NYC Citywide Payroll Data

### Meaghan Burke - Data 608

#### Data Source: https://data.cityofnewyork.us/City-Government/Citywide-Payroll-Data-Fiscal-Year-/k397-673e

### MAy 11th 2019


### Data Cleaning 
Steps:
1. Read in the raw_csv from the NYC Open Data website (download via link above)
2. Use script (view link below to access ipybn file) that does the following transformations:

    - Filtered for 'MANHATTAN', 'QUEENS', 'BRONX', 'BROOKLYN', & 'RICHMOND' boroughs

    - Filtered for full time, active employees 

    - Filtered for full year salaried employees

    - Removed null job titles

    - Removed 2014 as it is incomplete 

    - Converted all string values to uppercase & removed whitespace 

    - output the data to filtered_nyc_payset.csv. This is the base dataset for the dash application


In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [35]:
# all the filtering and data cleaning 
#filter for only per annum jobs, active employees and titles that are not null 
#del the original for memory purposes
borough_keep = ['MANHATTAN', 'QUEENS', 'BRONX', 'BROOKLYN', 'RICHMOND']
pay_dataset =  pd.read_csv("Citywide_Payroll_Data__Fiscal_Year_.csv", low_memory = False)
pay_dataset['Total Pay'] = pay_dataset[['Regular Gross Paid', 'Total OT Paid', 'Total Other Pay']].sum(axis =1 )
pay_dataset = pay_dataset.applymap(lambda s:s.upper().strip() if type(s) == str else s)
filtered_pay= pay_dataset[(pay_dataset['Pay Basis'] == 'PER ANNUM') & 
                          (pay_dataset['Leave Status as of June 30'] == 'ACTIVE') &
                          (~pay_dataset['Title Description'].isnull()) &
                          (pay_dataset['Fiscal Year'] != 2014) &
                          (pay_dataset['Work Location Borough'].isin(borough_keep))]

filtered_pay.loc[filtered_pay['Work Location Borough'].isnull(), 'Work Location Borough'] = 'UNKNOWN'
del(pay_dataset)

In [ ]:
#recreate a employee id as the Payroll Number has too many NAS, checked unqiue counts and the combination of the below columns is unqiue to each employee
#https://stackoverflow.com/questions/48008334/anonymize-specific-columns-with-pii-in-pandas-dataframe-python anonymize 
cols = ['Agency Name', 'Last Name', 'First Name', 'Mid Init', 'Agency Start Date', 'Pay Basis']
filtered_pay['Employee_Id'] = filtered_pay[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1).astype('category').cat.codes
filtered_pay.drop(['Last Name', 'Payroll Number', 'First Name', 'Mid Init', 'Leave Status as of June 30'], axis = 1, inplace = True)

In [ ]:
#this dataset will be used to source the dash application
filtered_pay.to_csv("filtered_nyc_payset.csv")

### Data EDA

In [87]:
filtered_pay.shape

(1113051, 14)

In [39]:
#display the unique descriptive information as a table in the dash application
unique_values = filtered_pay.groupby(['Fiscal Year']).nunique()

decribe_table = filtered_pay.describe()

unique_values

,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay,Total Pay,Employee_Id
Fiscal Year,,,,,,,,,,,,,,
2015,1,144,10448,5,1204,23503,1,11195,142723,25454,113948,110680,224996,264153
2016,1,144,10583,5,1206,23997,1,11749,129511,26704,114971,115618,213897,274049
2017,1,147,10685,5,1218,24413,1,13528,131364,25970,118924,116986,221903,285158
2018,1,148,10753,5,1236,23458,1,12790,131422,25127,119840,111806,227269,289435


In [40]:
decribe_table

,Fiscal Year,Base Salary,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay,Total Pay,Employee_Id
count,1.113051e+06,1.113051e+06,1.113051e+06,1.113051e+06,1.113051e+06,1.113051e+06,1.113051e+06,1.113051e+06,1.113051e+06
mean,2.016539e+03,6.936683e+04,1.128044e+03,6.684768e+04,1.059016e+02,5.790611e+03,3.732348e+03,7.637063e+04,1.832218e+05
std,1.115090e+00,2.716810e+04,9.422682e+02,2.906734e+04,1.862982e+02,1.113222e+04,6.205931e+03,3.632575e+04,1.041699e+05
min,2.015000e+03,1.000000e+00,-2.352500e+02,-1.871000e+02,-2.500000e+00,-2.649388e+04,-2.058165e+05,-2.422635e+04,0.000000e+00
25%,2.016000e+03,4.652000e+04,0.000000e+00,4.392921e+04,0.000000e+00,0.000000e+00,0.000000e+00,4.999988e+04,9.315900e+04
50%,2.017000e+03,6.800000e+04,1.790000e+03,6.519698e+04,0.000000e+00,0.000000e+00,4.448300e+02,7.164806e+04,1.829610e+05
75%,2.018000e+03,8.529200e+04,2.045720e+03,8.538705e+04,1.590000e+02,6.858325e+03,5.215790e+03,9.696892e+04,2.739740e+05
max,2.018000e+03,3.500000e+05,4.060920e+03,6.723089e+05,2.736670e+03,1.612902e+05,9.596096e+04,6.727313e+05,3.614010e+05


In [88]:
calcs = {'Work Location Borough': ['nunique'],'Employee_Id':['nunique'], 'Title Description':['nunique'],
        'Regular Hours': ['sum'],'Regular Gross Paid':['mean'], 'OT Hours':['sum'],'Total OT Paid' :['mean'],'Total Other Pay':['mean'],
        'Total Pay':['mean']}
consolidated_table = filtered_pay.groupby(['Agency Name']).agg(calcs).reset_index()
consolidated_table.columns = consolidated_table.columns.droplevel(-1)
consolidated_table = consolidated_table.sort_values('Title Description', ascending = False)

In [89]:
consolidated_table

,Agency Name,Work Location Borough,Employee_Id,Title Description,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay,Total Pay
70,DEPT OF HEALTH/MENTAL HYGIENE,5,6914,235,3.230388e+07,64074.741693,847082.26,1729.778457,2913.226616,68717.746766
120,POLICE DEPARTMENT,5,60824,235,3.913410e+08,70105.736092,47006951.94,13994.899833,9941.527080,94042.163005
110,NYC HOUSING AUTHORITY,5,10468,229,6.012044e+07,51306.009575,4201271.50,4482.342293,2040.936163,57829.288030
87,HRA/DEPT OF SOCIAL SERVICES,5,16289,185,9.058471e+07,51452.903157,3312592.25,2341.872774,3294.944208,57089.720138
60,DEPARTMENT OF EDUCATION ADMIN,5,14387,177,7.213618e+07,61702.686352,939122.96,1006.395935,2622.814544,65331.896831
69,DEPT OF ENVIRONMENT PROTECTION,5,4188,176,2.167631e+07,68638.323299,1463655.12,5400.133725,3035.329032,77073.786057
65,DEPARTMENT OF TRANSPORTATION,5,4604,167,2.436512e+07,63319.861456,2818752.97,10521.695843,3100.640023,76942.197322
84,FIRE DEPARTMENT,5,19905,152,1.302760e+08,72725.740803,21249128.98,20047.755669,9344.450146,102117.946618
0,ADMIN FOR CHILDREN'S SVCS,5,8464,145,4.140603e+07,59669.705000,3772626.62,6580.698694,2923.869115,69174.272809
59,DEPARTMENT OF CORRECTION,4,15307,140,8.269249e+07,64327.443459,15791479.34,20055.616780,7349.075491,91732.135730
